<a href="https://colab.research.google.com/github/js1022003/DL_study/blob/main/Cartoon_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"junseokimm","key":"14a0d44fd62b425b51fd6375c242ff13"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d volkandl/cartoon-classification

100% 11.5G/11.5G [02:34<00:00, 134MB/s] 
100% 11.5G/11.5G [02:34<00:00, 80.1MB/s]


In [ ]:
!unzip cartoon-classification.zip

In [ ]:
!pip install pillow
import tensorflow as tf
# import cupy as cp
# import cudf as cd
# import cuml as cl
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
BASE_DIR = '/content/cartoon_classification'
TRAIN_DIR = BASE_DIR+'/TRAIN/'
TEST_DIR = BASE_DIR+'/TEST/'
image_size = (256,256)

In [ ]:
def display_image(path):
    im = Image.open(path)
#     width,height = im.size
    im.show()
    display(im)

In [ ]:
def show_cartoon(number):
    for x in os.listdir(TRAIN_DIR):
        print(f"Image of cartoon {x}")
        path1 = TRAIN_DIR+x
        z = number
        for y in os.listdir(path1):
            if z<=0:
                break
            z-=1
            path2 = path1+'/'+y
            display_image(path2)

In [ ]:
# IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
inputs = tf.keras.Input(shape = (256,256,3))
x = base_model(inputs,training = False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024,activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024,activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(10,activation = 'softmax')(x)
model2 = tf.keras.Model(inputs, outputs)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip = True
                                  )
train_genator = train_datagen.flow_from_directory(TRAIN_DIR,batch_size = 512,target_size = (256,256))

Found 119617 images belonging to 10 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_genator = test_datagen.flow_from_directory(TEST_DIR,batch_size = 512,target_size = (256,256))

Found 18110 images belonging to 10 classes.


In [ ]:
model2.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.MeanAbsoluteError()]
             )

In [ ]:
history2 = model2.fit(train_genator,epochs = 5, verbose=1, validation_data = test_genator)

Epoch 1/5
234/234 [==============================] - 1324s 6s/step - loss: 0.0215 - accuracy: 0.9970 - precision_2: 0.9971 - recall_2: 0.9969 - mean_absolute_error: 6.3705e-04 - val_loss: 0.8770 - val_accuracy: 0.8760 - val_precision_2: 0.8825 - val_recall_2: 0.8738 - val_mean_absolute_error: 0.0254
Epoch 2/5
234/234 [==============================] - 1331s 6s/step - loss: 0.0214 - accuracy: 0.9972 - precision_2: 0.9973 - recall_2: 0.9972 - mean_absolute_error: 5.9030e-04 - val_loss: 1.1114 - val_accuracy: 0.9016 - val_precision_2: 0.9035 - val_recall_2: 0.9003 - val_mean_absolute_error: 0.0206
Epoch 3/5
234/234 [==============================] - 1341s 6s/step - loss: 0.0210 - accuracy: 0.9974 - precision_2: 0.9975 - recall_2: 0.9973 - mean_absolute_error: 5.6011e-04 - val_loss: 1.3598 - val_accuracy: 0.8279 - val_precision_2: 0.8401 - val_recall_2: 0.8263 - val_mean_absolute_error: 0.0344
Epoch 4/5
234/234 [==============================] - 1361s 6s/step - loss: 0.0145 - accuracy: 0.9

In [ ]:
loss, acc = model2.evaluate(test_genator, verbose=2)
print("test loss: %f, test accuracy: %f" %(loss, acc * 100))

36/36 - 228s - loss: 1.9610 - accuracy: 0.8606 - precision_2: 0.8702 - recall_2: 0.8569 - mean_absolute_error: 0.0286 - 228s/epoch - 6s/step


ValueError: ignored